In [1]:
from data.Safe2Unsafe import DeepAccidentDataset
from method.dynamics import InDCBFAttentionDynamics, InDCBFDynamics
from method.barriers import InDCBFAttentionBarrier, InDCBFBarrier
from method.dynamics import SABLASDynamics, SABLASUnFusedDynamics
from method.barriers import SABLASBarrier, SABLASBarrierUnfused
from method.trainers import InDCBFTrainer, SABLASTrainer
import torch
import time
import pytorch_lightning as pl
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
data = DeepAccidentDataset(train_batch_size=32,val_batch_size=32,num_workers=16)
data.setup()
train_dataloader = data.train_dataloader()
test_dataloader = data.test_dataloader()
latent_dim = 16

In [7]:
latent_dim = 16
backbones = ['vit','clip','vc1','resnet']
b2m = {'vit':"google/vit-base-patch16-224",'clip':"openai/clip-vit-base-patch16",'vc1':"vc1",'resnet':"resnet50"}
seeds = ['42']
# seeds = ['43','44','45','46']
for backbone in backbones:
    for seed in seeds:
        model_name = b2m[backbone]
        barrier = InDCBFAttentionBarrier(2,latent_dim=latent_dim)
        model = InDCBFAttentionDynamics(2,"cuda",model=model_name,latent_dim=latent_dim)
        trainer = InDCBFTrainer(model,barrier)
        checkpoint = torch.load(f"/root/autodl-tmp/logs/InDCBF_{backbone}_{seed}/version_0/checkpoints/last.ckpt")
        trainer.load_state_dict(checkpoint['state_dict'])
        trainer.to("cuda")
        a_all = []
        b_all = []
        label_all = []
        trainer.eval()
        for idx, (i,u,label) in enumerate(test_dataloader):
            i, u = i.to("cuda"), u.to("cuda")
            x,x_tide = trainer.model.simulate(i,u)
            b = trainer.barrier(x).squeeze(-1)
            b_all.append(b.cpu())
            label_all.append(label.squeeze(-1))

            d_b_safe = torch.autograd.grad(b.mean(),x,retain_graph=True)[0]
            with torch.no_grad():
                f, g = trainer.model.ode(x)
            gu = torch.einsum('btha,bta->bth',g.view(g.shape[0],g.shape[1],f.shape[-1],2),u)
            ascent_value = torch.einsum('bth,bth->bt', d_b_safe, (f + gu)) + b.squeeze(-1)
            a_all.append(ascent_value.cpu())
        import numpy as np
        asc = torch.cat(a_all)
        bs = torch.cat(b_all)
        labels = torch.cat(label_all)
        results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
        np.savetxt(f"./results_new/results_indcbf_{backbone}_{seed}.txt",results)
        results = torch.cat([asc,labels.unsqueeze(-1)],dim=-1).detach().numpy()
        np.savetxt(f"./results_new/results_indcbf_grad_{backbone}_{seed}.txt",results)

        # b_all = []
        # label_all = []
        # trainer.eval()
        # for idx, (i,u,label) in enumerate(test_dataloader):
        #     i, u = i.to("cuda"), u.to("cuda")
        #     x,x_tide = trainer.model.simulate(i,u)
        #     b = trainer.barrier(x)
        #     d_b_safe = torch.autograd.grad(b.mean(),x,retain_graph=True)[0]
        #     with torch.no_grad():
        #         f, g = trainer.model.ode(x)
        #     gu = torch.einsum('btha,bta->bth',g.view(g.shape[0],g.shape[1],f.shape[-1],2),u)
        #     ascent_value = torch.einsum('bth,bth->bt', d_b_safe, (f + gu)) + b.squeeze(-1)
        #     b_all.append(ascent_value.cpu())
        #     label_all.append(label.squeeze(-1))
        # bs = torch.cat(b_all)
        # labels = torch.cat(label_all)
        # results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
        # np.savetxt(f"./results_new/results_indcbf_grad_{backbone}_{seed}.txt",results)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 4/4 [00:00<00:00, 28.88it/s]


In [23]:
latent_dim = 16
backbones = ['vit','clip','vc1','resnet']
b2m = {'vit':"google/vit-base-patch16-224",'clip':"openai/clip-vit-base-patch16",'vc1':"vc1",'resnet':"resnet50"}
# seeds = ['42']
seeds = ['42','43','44','45','46']
for backbone in backbones:
    for seed in seeds:
        model_name = b2m[backbone]
        barrier = InDCBFBarrier(2,latent_dim=latent_dim)
        model = InDCBFDynamics(2,"cuda",model=model_name,latent_dim=latent_dim)
        trainer = InDCBFTrainer(model,barrier)
        checkpoint = torch.load(f"/root/autodl-tmp/logs/InDCBFUnfused_{backbone}_{seed}/version_0/checkpoints/last.ckpt")
        trainer.load_state_dict(checkpoint['state_dict'])
        trainer.to("cuda")
        a_all = []
        b_all = []
        label_all = []
        trainer.eval()
        for idx, (i,u,label) in enumerate(test_dataloader):
            i, u = i.to("cuda"), u.to("cuda")
            x,x_tide = trainer.model.simulate(i,u)
            b = trainer.barrier(x).squeeze(-1)
            b_all.append(b.cpu())
            label_all.append(label.squeeze(-1))

            B,T,N,H = x.shape
            weight = torch.einsum("btnh,btnh->btn",trainer.barrier.attention.expand(B,T,-1,-1),x)
            x_fused = torch.einsum("btn,btnh->btnh",weight,x).sum(2)
            b = trainer.barrier.cbf(x_fused)
            d_b_safe = torch.autograd.grad(b.mean(),x_fused,retain_graph=True)[0]
            with torch.no_grad():
                f, g = trainer.model.ode(x)
            gu = torch.einsum('btnha,btna->btnh',g.view(g.shape[0],g.shape[1],g.shape[2],f.shape[-1],2),u.unsqueeze(2).expand(-1,-1,g.shape[2],-1))
            dx = torch.einsum("btn,btnh->btnh",weight,(f + gu)).sum(2)
            ascent_value = torch.einsum('bth,bth->bt', d_b_safe, dx).unsqueeze(-1) + b
            a_all.append(ascent_value.cpu())
        import numpy as np
        asc = torch.cat(a_all)
        bs = torch.cat(b_all)
        labels = torch.cat(label_all)
        results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
        np.savetxt(f"./results_new/results_indcbfunfused_{backbone}_{seed}.txt",results)
        results = torch.cat([asc.squeeze(-1),labels.unsqueeze(-1)],dim=-1).detach().numpy()
        np.savetxt(f"./results_new/results_indcbfunfused_grad_{backbone}_{seed}.txt",results)

        # b_all = []
        # label_all = []
        # trainer.eval()
        # for idx, (i,u,label) in enumerate(test_dataloader):
        #     i, u = i.to("cuda"), u.to("cuda")
        #     x,x_tide = trainer.model.simulate(i,u)
        #     b = trainer.barrier(x)
        #     d_b_safe = torch.autograd.grad(b.mean(),x,retain_graph=True)[0]
        #     with torch.no_grad():
        #         f, g = trainer.model.ode(x)
        #     gu = torch.einsum('btha,bta->bth',g.view(g.shape[0],g.shape[1],f.shape[-1],2),u)
        #     ascent_value = torch.einsum('bth,bth->bt', d_b_safe, (f + gu)) + b.squeeze(-1)
        #     b_all.append(ascent_value.cpu())
        #     label_all.append(label.squeeze(-1))
        # bs = torch.cat(b_all)
        # labels = torch.cat(label_all)
        # results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
        # np.savetxt(f"./results_new/results_indcbf_grad_{backbone}_{seed}.txt",results)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 4/4 [00:00<00:00, 18.16it/s]
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 4/4 [00:00<00:00, 17.93it/s]
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 4/4 [00:0

FileNotFoundError: [Errno 2] No such file or directory: '/root/autodl-tmp/logs/InDCBFUnfused_clip_42/version_0/checkpoints/last.ckpt'

In [8]:
latent_dim = 16
# backbones = ['clip','vc1','resnet']
backbones = ['vit','clip','vc1','resnet']
b2m = {'vit':"google/vit-base-patch16-224",'clip':"openai/clip-vit-base-patch16",'vc1':"vc1",'resnet':"resnet50"}
seeds = ['42','43','44','45','46']
# seeds = ['46']
for backbone in backbones:
    for seed in seeds:
        model_name = b2m[backbone]
        barrier = SABLASBarrier(2,latent_dim=latent_dim)
        model = SABLASDynamics(2,"cuda",model=model_name,latent_dim=latent_dim)
        trainer = SABLASTrainer(model,barrier)
        checkpoint = torch.load(f"/root/tf-logs/SABLAS_{backbone}_{seed}/version_0/checkpoints/last.ckpt")
        trainer.load_state_dict(checkpoint['state_dict'])
        trainer.to("cuda")
        a_all = []
        b_all = []
        label_all = []
        trainer.eval()
        for idx, (i,u,label) in enumerate(test_dataloader):
            i, u = i.to("cuda"), u.to("cuda")
            x,x_tide = trainer.model.simulate(i,u)
            b = trainer.barrier(x).squeeze(-1)
            b_all.append(b.cpu())
            label_all.append(label.squeeze(-1))

            x_tide = x[:,1:]
            x = x[:,:-1]
            b = trainer.barrier(x)
            f,g = trainer.model.ode(x)
            gu = torch.einsum('btha,bta->bth',g.view(g.shape[0],g.shape[1],f.shape[-1],2),u[:,:-1])
            derive = f + gu
            x_nom = x + derive*0.1
            x_nom = x_nom + (x_tide-x_nom).detach()
            ascent_value = b + (trainer.barrier(x_nom)-b)/0.1
            a_all.append(ascent_value.cpu())
        import numpy as np
        asc = torch.cat(a_all)
        bs = torch.cat(b_all)
        labels = torch.cat(label_all)
        results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
        np.savetxt(f"./results_new/results_sablas_{backbone}_{seed}.txt",results)
        results = torch.cat([asc.squeeze(-1),labels.unsqueeze(-1)],dim=-1).detach().numpy()
        np.savetxt(f"./results_new/results_sablas_grad_{backbone}_{seed}.txt",results)

        # b_all = []
        # label_
        # for idx, (i,u,label) in enumerate(test_dataloader):
        #     ia"), u.to("cuda")
        #     x,x_tide = trainer.model.simulate(i,u)
        #     x_tide = x[:,1:]
        #     x = x[:,:-1]
        #     b = trainer.barrier(x)
        #     f,g = trainer.model.ode(x)
        #     gu = torch.einsum('btha,bta->bth',g.view(g.shape[0],g.shape[1],f.shape[-1],2),u[:,:-1])
        #     derive = f + gu
        #     x_nom = x + derive*0.1
        #     x_nom = x_nom + (x_tide-x_nom).detach()
        #     ascent_value = b + (trainer.barrier(x_nom)-b)/0.1
        #     b_all.append(ascent_value.cpu())
        #     label_all.append(label.squeeze(-1))
        # bs = torch.cat(b_all)
        # labels = torch.cat(label_all)
        # results = torch.cat([bs.squeeze(-1),labels.unsqueeze(-1)],dim=-1).detach().numpy()
        # np.savetxt(f"./res/results_sablasunfused_grad_{backbone}_{seed}.txt",results)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 4/4 [00:00<00:00, 22.24it/s]
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 4/4 [00:00<00:00, 21.97it/s]
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 4/4 [00:0

In [16]:
latent_dim = 16
backbones = ['vit']
# backbones = ['vit','clip','vc1','resnet']
b2m = {'vit':"google/vit-base-patch16-224",'clip':"openai/clip-vit-base-patch16",'vc1':"vc1",'resnet':"resnet50"}
# seeds = ['43','44','45','46']
seeds = ['42']
for backbone in backbones:
    for seed in seeds:
        model_name = b2m[backbone]
        barrier = SABLASBarrierUnfused(2,latent_dim=latent_dim)
        model = SABLASUnFusedDynamics(2,"cuda",model=model_name,latent_dim=latent_dim)
        trainer = SABLASTrainer(model,barrier)
        checkpoint = torch.load(f"/root/tf-logs/SABLASUnfused_{backbone}_{seed}/version_0/checkpoints/last.ckpt")
        trainer.load_state_dict(checkpoint['state_dict'])
        trainer.to("cuda")
        a_all = []
        b_all = []
        label_all = []
        trainer.eval()
        for idx, (i,u,label) in enumerate(test_dataloader):
            i, u = i.to("cuda"), u.to("cuda")
            x,x_tide = trainer.model.simulate(i,u)
            b = trainer.barrier(x).squeeze(-1)
            b_all.append(b.cpu())
            label_all.append(label.squeeze(-1))

            x_tide = x[:,1:]
            x = x[:,:-1]
            b = trainer.barrier(x)
            f,g = trainer.model.ode(x)
            gu = torch.einsum('btnha,btna->btnh',g.view(g.shape[0],g.shape[1],g.shape[2],f.shape[-1],2),u[:,:-1].unsqueeze(2).expand(-1,-1,g.shape[2],-1))
            derive = f + gu
            x_nom = x + derive*0.1
            x_nom = x_nom + (x_tide-x_nom).detach()
            ascent_value = b + (trainer.barrier(x_nom)-b)/0.1
            a_all.append(ascent_value.cpu())
        import numpy as np
        asc = torch.cat(a_all)
        bs = torch.cat(b_all)
        labels = torch.cat(label_all)
        results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
        np.savetxt(f"./results_new/results_sablasunfused_{backbone}_{seed}.txt",results)
        results = torch.cat([asc.squeeze(-1),labels.unsqueeze(-1)],dim=-1).detach().numpy()
        np.savetxt(f"./results_new/results_sablasunfsued_grad_{backbone}_{seed}.txt",results)

        # b_all = []
        # label_all = []
        # trainer.eval()
        # for idx, (i,u,label) in enumerate(test_dataloader):
        #     i, u = i.to("cuda"), u.to("cuda")
        #     x,x_tide = trainer.model.simulate(i,u)
        #     x_tide = x[:,1:]
        #     x = x[:,:-1]
        #     b = trainer.barrier(x)
        #     f,g = trainer.model.ode(x)
        #     gu = torch.einsum('btnha,btna->btnh',g.view(g.shape[0],g.shape[1],g.shape[2],f.shape[-1],2),u[:,:-1].unsqueeze(2).expand(-1,-1,g.shape[2],-1))
        #     derive = f + gu
        #     x_nom = x + derive*0.1
        #     x_nom = x_nom + (x_tide-x_nom).detach()
        #     ascent_value = b + (trainer.barrier(x_nom)-b)/0.1
        #     b_all.append(ascent_value.cpu())
        #     label_all.append(label.squeeze(-1))
        # bs = torch.cat(b_all)
        # labels = torch.cat(label_all)
        # results = torch.cat([bs.squeeze(-1),labels.unsqueeze(-1)],dim=-1).detach().numpy()
        # np.savetxt(f"./res/results_sablasunfused_grad_{backbone}_{seed}.txt",results)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for SABLASTrainer:
	Missing key(s) in state_dict: "model.encoder.ViT.embeddings.cls_token", "model.encoder.ViT.embeddings.position_embeddings", "model.encoder.ViT.embeddings.patch_embeddings.projection.weight", "model.encoder.ViT.embeddings.patch_embeddings.projection.bias", "model.encoder.ViT.encoder.layer.0.attention.attention.query.weight", "model.encoder.ViT.encoder.layer.0.attention.attention.query.bias", "model.encoder.ViT.encoder.layer.0.attention.attention.key.weight", "model.encoder.ViT.encoder.layer.0.attention.attention.key.bias", "model.encoder.ViT.encoder.layer.0.attention.attention.value.weight", "model.encoder.ViT.encoder.layer.0.attention.attention.value.bias", "model.encoder.ViT.encoder.layer.0.attention.output.dense.weight", "model.encoder.ViT.encoder.layer.0.attention.output.dense.bias", "model.encoder.ViT.encoder.layer.0.intermediate.dense.weight", "model.encoder.ViT.encoder.layer.0.intermediate.dense.bias", "model.encode

In [14]:
asc.shape

torch.Size([3216, 4, 1])